In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                               VALID:        \
       Time   Ep   Ct     LOSS   PPL      NLL     KL   REG     LOSS   PPL   
5000     79    8  463  9179.00  1881  9160.66  18.09  0.25  9157.78  1795   
10000    76   17  360  9138.35  1819  9118.60  19.56  0.20  9141.21  1772   
15000    77   26  257  9119.68  1791  9099.47  20.03  0.18  9136.41  1764   
20000    82   35  154  9108.40  1775  9087.96  20.24  0.17  9131.93  1758   
25000    77   44   51  9100.47  1764  9079.91  20.35  0.16  9128.96  1753   
30000    76   52  515  9094.68  1755  9074.06  20.42  0.15  9130.43  1755   
35000    76   61  412  9090.26  1749  9069.71  20.47  0.14  9127.13  1750   
40000    76   70  309  9086.61  1744  9066.14  20.50  0.14  9126.26  1749   
45000    76   79  206  9083.46  1739  9063.04  20.52  0.14  9124.95  1747   
50000    77   88  103  9080.78  1735  9060.37  20.54  0.13  9123.37  1745   
55000    76   97    0  9078.40  1732  9057.97  20.55  0.13  9124.27  1746   
60000    77  105  464  9076.45  1729  9056.00  20.55  0.13  9123.53  1744   
65000    77  114  361  9074.92  1727  9054.33  20.56  0.12  9122.34  1743   
70000    75  123  258  9073.48  1724  9052.76  20.57  0.12  9122.49  1743   
75000    98  132  155  9072.12  1722  9051.33  20.57  0.12  9121.43  1741   
80000    77  141   52  9070.83  1721  9049.98  20.57  0.12  9121.63  1742   
85000    77  149  516  9069.71  1719  9048.80  20.57  0.12  9122.34  1742   
90000    77  158  413  9068.72  1717  9047.79  20.58  0.12  9120.39  1740   
95000    77  167  310  9067.82  1716  9046.85  20.58  0.11  9120.42  1739   
100000   77  176  207  9066.93  1715  9045.93  20.58  0.11  9119.93  1739   
105000   78  185  104  9066.11  1713  9045.08  20.58  0.11  9118.20  1737   
110000   78  194    1  9065.28  1712  9044.22  20.59  0.11  9120.85  1740   
115000   78  202  465  9064.58  1711  9043.48  20.59  0.11  9119.99  1739   
120000   77  211  362  9063.93  1710  9042.85  20.59  0.11  9119.73  1739   
125000   79  220  259  9063.23  1709  9042.27  20.59  0.11  9120.27  1739   
130000   79  229  156  9062.55  1708  9041.70  20.60  0.11  9118.46  1737   
135000   77  238   53  9061.87  1708  9041.12  20.60  0.11  9119.31  1738   
140000   77  246  517  9061.26  1707  9040.60  20.60  0.11  9120.30  1739   
145000   78  255  414  9060.73  1706  9040.14  20.60  0.11  9119.67  1739   
150000   77  264  311  9060.24  1705  9039.71  20.60  0.10  9119.81  1739   
155000   78  273  208  9059.74  1705  9039.27  20.61  0.10  9119.59  1738   
160000   78  282  105  9059.26  1704  9038.85  20.61  0.10  9118.99  1738   
165000   77  291    2  9058.77  1703  9038.40  20.61  0.10  9120.76  1740   

                                TEST:       SPEC:             HIER:        
            NLL     KL   REG     LOSS   PPL     1     2     3 CHILD OTHER  
5000    9137.22  20.39  0.17  9158.85  1797  0.39  0.37  0.44  0.40  0.28  
10000   9120.32  20.75  0.14  9141.49  1772  0.41  0.40  0.47  0.37  0.25  
15000   9115.47  20.80  0.14  9136.27  1764  0.43  0.41  0.48  0.36  0.25  
20000   9111.07  20.73  0.12  9132.58  1759  0.44  0.42  0.48  0.34  0.24  
25000   9108.36  20.48  0.12  9129.66  1755  0.45  0.42  0.47  0.35  0.25  
30000   9109.65  20.66  0.12  9129.66  1755  0.46  0.42  0.47  0.36  0.25  
35000   9106.54  20.48  0.11  9126.85  1749  0.46  0.42  0.47  0.35  0.25  
40000   9105.65  20.50  0.11  9125.06  1748  0.46  0.43  0.47  0.35  0.25  
45000   9104.35  20.50  0.10  9124.50  1746  0.46  0.43  0.48  0.34  0.24  
50000   9102.73  20.54  0.10  9123.78  1746  0.46  0.44  0.48  0.33  0.24  
55000   9103.76  20.40  0.10  9123.78  1746  0.47  0.43  0.48  0.34  0.24  
60000   9103.13  20.30  0.10  9123.91  1745  0.46  0.43  0.47  0.35  0.25  
65000   9101.83  20.41  0.09  9122.44  1744  0.46  0.43  0.48  0.34  0.24  
70000   9101.94  20.46  0.10  9122.24  1743  0.46  0.44  0.47  0.34  0.24  
75000   9100.91  20.43  0.09  9121.05  1740  0.46  0.44  0.48  0.34  0.24  
80000   9101.06  20.47  0.09  9121.0

0 R: 1.000 P: 0.119 user dialogue speech users utterance systems utterances research knowledge domain
   1 R: 0.238 P: 0.061 morphological pos languages rules analysis tags forms form annotation english
     11 R: 0.097 P: 0.097 verb structure semantic verbs syntactic noun object type subject clause
     12 R: 0.079 P: 0.079 tree parsing grammar parser dependency node trees rules parse structure
   2 R: 0.142 P: 0.043 extraction entity patterns names domain entities named precision terms pattern
     21 R: 0.049 P: 0.049 semantic sense lexical wordnet senses relations concepts knowledge concept frame
     22 R: 0.049 P: 0.049 event relation relations entity discourse annotation events entities argument features
   3 R: 0.137 P: 0.045 document documents topic question query sentences questions answer search web
     31 R: 0.026 P: 0.026 english alignment languages translation parallel pairs chinese corpora bilingual source
     32 R: 0.066 P: 0.066 similarity graph pairs vector context 

In [10]:
config.dim_bow

5424